In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# !pip install pyjwt==1.6.4
import os
import jwt
import uuid
import hashlib
from urllib.parse import urlencode
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import copy

In [3]:
coins = ['KRW-BTC','KRW-ETH','KRW-XRP']
#coins = ['KRW-ETH']
df_coin = {}
for coin in coins:
    print(coin)
    df_coin[coin] = getCoinDF(coin,30,200)
  
df_coin['KRW-ETH']

KRW-BTC
KRW-ETH
KRW-XRP


ValueError: Expected object or value

In [81]:
coin_name = 'KRW-ETH'
df = df_coin[coin_name]
df = df.loc[::-1,:]
df = df.reset_index()
df

,index,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,999,KRW-ETH,2021-05-10T21:00:00,2021-05-11T06:00:00,5049000,5115000,4995000,5064000,2021-05-10 21:30:00.138,4.550917e+10,8978.386773,30
1,998,KRW-ETH,2021-05-10T21:30:00,2021-05-11T06:30:00,5068000,5170000,5023000,5147000,2021-05-10 22:00:00.287,4.322203e+10,8490.874198,30
2,997,KRW-ETH,2021-05-10T22:00:00,2021-05-11T07:00:00,5147000,5200000,5112000,5150000,2021-05-10 22:29:59.988,4.094357e+10,7925.304005,30
3,996,KRW-ETH,2021-05-10T22:30:00,2021-05-11T07:30:00,5150000,5155000,5097000,5138000,2021-05-10 22:59:59.088,2.979543e+10,5814.918994,30
4,995,KRW-ETH,2021-05-10T23:00:00,2021-05-11T08:00:00,5140000,5141000,4950000,4976000,2021-05-10 23:30:00.461,6.211992e+10,12326.948811,30
...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,KRW-ETH,2021-05-31T12:30:00,2021-05-31T21:30:00,3003000,3004000,2960000,2985000,2021-05-31 12:59:58.492,3.256290e+10,10934.905259,30
996,3,KRW-ETH,2021-05-31T13:00:00,2021-05-31T22:00:00,2985000,3027000,2968000,3011000,2021-05-31 13:29:58.487,2.010353e+10,6708.953051,30
997,2,KRW-ETH,2021-05-31T13:30:00,2021-05-31T22:30:00,3010000,3015000,2970000,2979000,2021-05-31 13:59:56.968,1.852639e+10,6197.938155,30
998,1,KRW-ETH,2021-05-31T14:00:00,2021-05-31T23:00:00,2980000,2991000,2946000,2969000,2021-05-31 14:29:59.779,1.850617e+10,6237.424095,30


In [105]:
df['opening_price_bef'] = df['opening_price'].shift(1)
df['opening_price_bef2'] = df['opening_price'].shift(2)
df['opening_price_after'] = df['opening_price'].shift(-1)
df['diff'] = df['opening_price']-df['opening_price_bef']
df['diff2'] = df['opening_price_bef']-df['opening_price_bef2']
df['diff_after'] = df['opening_price_after']-df['opening_price']
df.fillna(0,inplace=True)
df['diff_up'] = df.apply(lambda x : 1 if x['diff']>=0 and x['diff2']>=0 else 0,axis=1)
df[['opening_price','opening_price_bef','opening_price_after','diff','diff2','diff_after','diff_up']]

,opening_price,opening_price_bef,opening_price_after,diff,diff2,diff_after,diff_up
0,5049000,0.0,5068000.0,0.0,0.0,19000.0,1
1,5068000,5049000.0,5147000.0,19000.0,0.0,79000.0,1
2,5147000,5068000.0,5150000.0,79000.0,19000.0,3000.0,1
3,5150000,5147000.0,5140000.0,3000.0,79000.0,-10000.0,1
4,5140000,5150000.0,4980000.0,-10000.0,3000.0,-160000.0,0
...,...,...,...,...,...,...,...
995,3003000,3003000.0,2985000.0,0.0,-49000.0,-18000.0,0
996,2985000,3003000.0,3010000.0,-18000.0,0.0,25000.0,0
997,3010000,2985000.0,2980000.0,25000.0,-18000.0,-30000.0,0
998,2980000,3010000.0,2969000.0,-30000.0,25000.0,-11000.0,0


In [104]:
df[['diff_up','diff_after']].groupby(['diff_up']).mean()

,diff_after
diff_up,
0,-1400.000000
1,-4356.521739


In [87]:
money = 1000000
stock = 0
write = []

for i in range(len(df)):
    diff = df['diff'].loc[i]
    if diff<0:
        if money>1:
            stock += money/df['opening_price'].loc[i]
            money = 0
            continue

    if stock>0:
        money += df['opening_price'].loc[i]*stock
        stock = 0
        continue

    # if diff>0:
    #     if stock>0:
    #         money += df['opening_price'].loc[i]*stock
    #         stock = 0

    write.append(money+stock*df['opening_price'].loc[i])

print(money+stock*df['opening_price'].loc[i])
#write[:10]

968549.3864727891


## tf agent 이용한 PPO 모델 사용

In [5]:
# import os
# from datetime import datetime
# from collections import deque
# import numpy as np

# base_pwd = "C://Users/cih46/Desktop/bit"

# def Write_to_file(Date, net_worth, filename='{}.txt'.format(datetime.now().strftime("%m_%d_%H_%M_%S"))):
#     for i in net_worth: 
#         Date += " {}".format(i)
#     #print(Date)
#     if not os.path.exists('logs'):
#         os.makedirs('logs')
#     file = open(base_pwd+"/logs/"+filename, 'a+')
#     file.write(Date+"\n")
#     file.close()

In [6]:
df = df['KRW-ETH']
df.rename(columns={'opening_price' : 'Open','trade_price':'Close','high_price' : 'High', 'candle_date_time_kst' : 'Date',
'candle_acc_trade_volume':'Volume','low_price':'Low'}, inplace = True)
df = df[['Date','Open','High','Low','Close','Volume']]
df

,Date,Open,High,Low,Close,Volume
0,2021-04-28T01:30:00,3087000,3100000,3069000,3073000,1655.638545
1,2021-04-28T02:00:00,3073000,3087000,3055000,3077000,1555.382445
2,2021-04-28T02:30:00,3080000,3094000,3076000,3088000,570.217042
3,2021-04-28T03:00:00,3088000,3089000,3071000,3078000,271.719260
4,2021-04-28T03:30:00,3079000,3091000,3076000,3090000,246.889809
...,...,...,...,...,...,...
995,2021-05-18T17:00:00,4287000,4319000,4283000,4310000,4529.736673
996,2021-05-18T17:30:00,4310000,4310000,4250000,4255000,4884.569741
997,2021-05-18T18:00:00,4261000,4273000,4208000,4232000,4408.915316
998,2021-05-18T18:30:00,4232000,4254000,4225000,4248000,2636.969311


In [7]:
from collections import defaultdict
import random

class QLearningAgent:
    def __init__(self, actions):
        # 행동 = [0, 1, 2] 순서대로 hold, buy, sell
        self.actions = actions
        self.learning_rate = 0.01
        self.discount_factor = 0.99
        self.epsilon = 0.9
        self.q_table = defaultdict(lambda : [0,0,0])

    def get_q(self,state) :
        string_state = format(state,".1f")

        return self.q_table[string_state]        


    def learn(self, state, action, reward, next_state):
        q_1 = self.get_q(state)[action]
        # 벨만 최적 방정식을 사용한 큐함수의 업데이트
        q_2 = reward + self.discount_factor * max(self.get_q(next_state))        
        self.get_q(state)[action] += self.learning_rate * (q_2 - q_1)

    # 큐함수에 의거하여 입실론 탐욕 정책에 따라서 행동을 반환
    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            # 무작위 행동 반환
            action = np.random.choice(self.actions)
        else:
            # 큐함수에 따른 행동 반환
            state_action = self.get_q(state)
            action = self.arg_max(state_action)
        
        self.epsilon *= 1.001
        return action        
    def get_random_action(self,state) :
        action = np.random.choice(self.actions)
        return action

    @staticmethod
    def arg_max(state_action):
        max_index_list = []
        max_value = state_action[0]
        for index, value in enumerate(state_action):
            if value > max_value:
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
            elif value == max_value:
                max_index_list.append(index)
        return random.choice(max_index_list)

In [8]:
def q_learning(agent,env, train_episodes = 1,is_train=False,is_random=False):
    
    average_net_worth = 0

    for episode in range(train_episodes):
        state = env.reset()

        while True:

            env.render()

            action = agent.get_action(state)
            if is_random:
                action = agent.get_random_action(state)

            next_state, reward, done = env.step(action)

            agent.learn(state, action, reward, next_state)
            
            state = next_state

            if env.current_step == env.end_step:
                average_net_worth += env.net_worth
                print("net_worth:", env.net_worth)
                print(agent.q_table)
                break
            
            if done:
                break     

    print("average_net_worth:", average_net_worth/train_episodes)

In [9]:
import customEnv as CustomEnv
test_size = -400
train_df = df[:test_size]
test_df = df[test_size:].reset_index()

train_df

,Date,Open,High,Low,Close,Volume
0,2021-04-28T01:30:00,3087000,3100000,3069000,3073000,1655.638545
1,2021-04-28T02:00:00,3073000,3087000,3055000,3077000,1555.382445
2,2021-04-28T02:30:00,3080000,3094000,3076000,3088000,570.217042
3,2021-04-28T03:00:00,3088000,3089000,3071000,3078000,271.719260
4,2021-04-28T03:30:00,3079000,3091000,3076000,3090000,246.889809
...,...,...,...,...,...,...
595,2021-05-10T10:00:00,4777000,4777000,4751000,4760000,2022.883517
596,2021-05-10T10:30:00,4757000,4815000,4756000,4780000,2339.488664
597,2021-05-10T11:00:00,4780000,4843000,4779000,4820000,3135.806979
598,2021-05-10T11:30:00,4820000,4868000,4810000,4862000,4264.784991


In [10]:
test_df

,index,Date,Open,High,Low,Close,Volume
0,600,2021-05-10T12:00:00,4864000,4932000,4861000,4908000,11896.961451
1,601,2021-05-10T12:30:00,4908000,4946000,4900000,4914000,8223.173668
2,602,2021-05-10T13:00:00,4914000,4927000,4887000,4912000,5437.053095
3,603,2021-05-10T13:30:00,4911000,4983000,4911000,4981000,7363.056358
4,604,2021-05-10T14:00:00,4981000,5016000,4941000,4970000,15615.365537
...,...,...,...,...,...,...,...
395,995,2021-05-18T17:00:00,4287000,4319000,4283000,4310000,4529.736673
396,996,2021-05-18T17:30:00,4310000,4310000,4250000,4255000,4884.569741
397,997,2021-05-18T18:00:00,4261000,4273000,4208000,4232000,4408.915316
398,998,2021-05-18T18:30:00,4232000,4254000,4225000,4248000,2636.969311


In [11]:
for i in range(100):
    print( round((train_df.loc[i,'Close'] - train_df.loc[i, 'Open'])*100  / train_df.loc[i, 'Open'] , 1) )

-0.5
0.1
0.3
-0.3
0.4
-0.8
0.4
0.2
0.4
-0.4
0.5
-0.4
-0.3
0.5
1.2
0.8
0.3
-0.0
0.3
-0.5
-0.8
-0.8
-1.4
0.8
-1.1
-0.5
-0.1
-1.4
0.7
1.3
-0.8
0.3
-1.2
-0.0
1.2
1.1
0.3
1.4
0.5
0.9
-0.0
-0.9
0.0
0.5
-0.8
-0.7
-0.1
0.8
0.5
0.1
-0.4
0.4
-0.0
1.0
-0.1
-0.8
0.4
0.5
1.2
-1.1
-0.6
0.3
0.9
-1.1
-0.6
0.1
1.4
0.1
-0.1
-0.2
-0.5
-0.5
0.0
0.8
0.3
0.0
0.4
-0.3
0.2
-0.6
0.4
1.1
0.2
-0.2
0.3
-0.2
-0.1
1.5
-0.1
-0.4
-1.1
0.8
-0.1
0.7
-0.2
-0.7
-0.2
0.2
0.1
-0.4


In [12]:
train_env = CustomEnv.CustomEnv(df=train_df)
test_env = CustomEnv.CustomEnv(df=test_df)

agent = QLearningAgent(actions=list(range(len(train_env.action_space))))

q_learning(agent,train_env, train_episodes = 1,is_train=True)
q_learning(agent,test_env, train_episodes = 1,is_train=False)

585643
Step: 513, Net Worth: 1122.990406803605
Step: 514, Net Worth: 1122.990406803605
Step: 515, Net Worth: 1130.1196663812439
Step: 516, Net Worth: 1130.1196663812439
Step: 517, Net Worth: 1126.4195552190977
Step: 518, Net Worth: 1126.4195552190977
Step: 519, Net Worth: 1126.4195552190977
Step: 520, Net Worth: 1126.4195552190977
Step: 521, Net Worth: 1123.769156265641
Step: 522, Net Worth: 1127.744754695826
Step: 523, Net Worth: 1146.8276271607144
Step: 524, Net Worth: 1163.2601006721459
Step: 525, Net Worth: 1164.3202602535287
Step: 526, Net Worth: 1164.3202602535287
Step: 527, Net Worth: 1166.9791327364096
Step: 528, Net Worth: 1166.9791327364096
Step: 529, Net Worth: 1206.0031715315347
Step: 530, Net Worth: 1244.4926344527537
Step: 531, Net Worth: 1244.4926344527537
Step: 532, Net Worth: 1244.4926344527537
Step: 533, Net Worth: 1244.4926344527537
Step: 534, Net Worth: 1244.4926344527537
Step: 535, Net Worth: 1244.4926344527537
Step: 536, Net Worth: 1246.6908992078038
Step: 537, Ne